In [1]:
import itertools

In [9]:
FDIC = pd.DataFrame(pd.read_csv("../data/ACdata.csv"))
print(FDIC.shape)
FDIC['state_abv'] = list(map(lambda x: x[-2:], FDIC['county_state']))
#states = ["AL","AK","FL","GA","LA","MD","MS","NC","SC","TN","TX"]
#FDIC = FDIC[FDIC['state_abv'].isin(states)]
print(FDIC.shape)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(653106, 40)
(653106, 41)


In [10]:
FDIC.columns

Index(['FIPS_county', 'year', 'cert', 'county_state', 'state', 'FIPS_state',
       'pop', 'bankname', 'cpi_2018', 'deposits', 'branch_count', 'cb',
       'asset', 'all_loans', 'equity', 'real_estate_loans', 'ag_loans',
       'ci_loans', 'consumer_loans', 'credit_card_loans', 'na_asset',
       'na_real_estate_loans', 'na_ag_loans', 'na_ci_loans',
       'na_consumer_loans', 'na_credit_card_loans',
       'dollarloans_nonfarm_bus_LT1M', 'dollarloans_ci_LT1M',
       'dollarloans_ag_bus_LT500K', 'numloans_nonfarm_bus_LT1M',
       'numloans_ci_LT1M', 'numloans_ag_bus_LT500K', 'percap_pers_inc', 'jobs',
       'farm_inc', 'farm_jobs', 'nonfarm_inc', 'nonfarm_jobs', 'owner_inc',
       'owner_jobs', 'state_abv'],
      dtype='object')

In [11]:
sb_cols = ['FIPS_county', 'county_state', 'year', 'pop', 'cpi_2018', 'cb', 'branch_count', 'deposits', 'asset', 'all_loans', 'equity',
           'ci_loans', 'credit_card_loans', 'na_asset', 'na_ci_loans', 'na_credit_card_loans',
           'dollarloans_nonfarm_bus_LT1M', 'dollarloans_ci_LT1M','numloans_nonfarm_bus_LT1M',
           'numloans_ci_LT1M', 'percap_pers_inc', 'jobs','farm_inc', 'farm_jobs', 'nonfarm_inc', 'nonfarm_jobs', 'owner_inc',
       'owner_jobs']
FDIC_sb = FDIC[sb_cols]
FDIC_sb.head()

,FIPS_county,county_state,year,pop,cpi_2018,cb,branch_count,deposits,asset,all_loans,...,numloans_nonfarm_bus_LT1M,numloans_ci_LT1M,percap_pers_inc,jobs,farm_inc,farm_jobs,nonfarm_inc,nonfarm_jobs,owner_inc,owner_jobs
0,1001,"Autauga, AL",1994,38186.0,58.883015,1,3,10861,288171,141793,...,NaN,NaN,18683.0,12751.0,7521.0,424.0,41002.0,2519.0,48523.0,2943.0
1,1001,"Autauga, AL",1994,38186.0,58.883015,0,1,27982,9709521,5933900,...,NaN,NaN,18683.0,12751.0,7521.0,424.0,41002.0,2519.0,48523.0,2943.0
2,1001,"Autauga, AL",1994,38186.0,58.883015,0,2,23991,8219440,5740534,...,NaN,NaN,18683.0,12751.0,7521.0,424.0,41002.0,2519.0,48523.0,2943.0
3,1001,"Autauga, AL",1994,38186.0,58.883015,1,3,121131,160392,87651,...,NaN,NaN,18683.0,12751.0,7521.0,424.0,41002.0,2519.0,48523.0,2943.0
4,1001,"Autauga, AL",1994,38186.0,58.883015,0,2,17250,499998,337683,...,NaN,NaN,18683.0,12751.0,7521.0,424.0,41002.0,2519.0,48523.0,2943.0


In [12]:
FDIC_sb_i = FDIC_sb.reset_index(drop = True)

In [13]:
print("Number of missing values in na_credit_card_loans column is", sum(FDIC_sb['na_credit_card_loans'].isna()))
print("Number of missing values in dollarloans_nonfarm_bus_LT1M column is", sum(FDIC_sb['dollarloans_nonfarm_bus_LT1M'].isna()))
print("Number of missing values in dollarloans_ci_LT1M column is", sum(FDIC_sb['dollarloans_ci_LT1M'].isna()))
print("Number of missing values in numloans_nonfarm_bus_LT1M column is", sum(FDIC_sb['numloans_nonfarm_bus_LT1M'].isna()))
print("Number of missing values in numloans_ci_LT1M column is", sum(FDIC_sb['numloans_ci_LT1M'].isna()))

Number of missing values in na_credit_card_loans column is 12798
Number of missing values in dollarloans_nonfarm_bus_LT1M column is 97376
Number of missing values in dollarloans_ci_LT1M column is 97376
Number of missing values in numloans_nonfarm_bus_LT1M column is 97376
Number of missing values in numloans_ci_LT1M column is 97376


In [14]:
#eliminating null values, adjusting income 
FDIC_sb_i = FDIC_sb_i[~FDIC_sb_i['na_credit_card_loans'].isna()].fillna(0)
FDIC_sb_i = FDIC_sb_i[~FDIC_sb_i['dollarloans_ci_LT1M'].isna()].fillna(0)

In [15]:
keep = ["FIPS_county", "year", "pop", "cpi_2018","jobs", "percap_pers_inc", "cb", ]
sums = ["branch_count",'deposits', 'asset', 'all_loans', 'equity',
           'ci_loans', 'credit_card_loans', 'na_asset', 'na_ci_loans', 'na_credit_card_loans',
           'dollarloans_nonfarm_bus_LT1M', 'dollarloans_ci_LT1M','numloans_nonfarm_bus_LT1M',
           'numloans_ci_LT1M','farm_inc', 'farm_jobs', 'nonfarm_inc', 'nonfarm_jobs', 'owner_inc',
       'owner_jobs']

FDIC_sb_gi = FDIC_sb_i.groupby(keep)[sums].sum()
print(FDIC_sb_gi.shape)
FDIC_sb_gi

(134352, 20)


branch_count  \
FIPS_county year pop     cpi_2018   jobs    percap_pers_inc cb                 
1001        1994 38186.0 58.883015  12751.0 18683.0         0              5   
                                                            1              6   
            1995 39112.0 60.674587  13033.0 19350.0         0              5   
                                                            1              6   
            1996 40207.0 62.386536  13644.0 20155.0         0              5   
...                                                                      ...   
56045       2016 7213.0  95.607864  4043.0  42585.0         1              2   
            2017 6986.0  97.214310  3964.0  42816.0         0              1   
                                                            1              2   
            2018 6967.0  100.000000 4043.0  44737.0         0              1   
                                                            1              2   

                                                                deposits  \
FIPS_county year pop     cpi_2018   jobs    percap_pers_inc cb             
1001        1994 38186.0 58.883015  12751.0 18683.0         0      69223   
                                                            1     131992   
            1995 39112.0 60.674587  13033.0 19350.0         0      88316   
                                                            1     169776   
            1996 40207.0 62.386536  13644.0 20155.0         0     101855   
...                                                                  ...   
56045       2016 7213.0  95.607864  4043.0  42585.0         1     152963   
            2017 6986.0  97.214310  3964.0  42816.0         0      66557   
                                                            1     149275   
            2018 6967.0  100.000000 4043.0  44737.0         0      71540   
                                                            1     144001   

                                                                   asset  \
FIPS_county year pop     cpi_2018   jobs    percap_pers_inc cb             
1001        1994 38186.0 58.883015  12751.0 18683.0         0   18428959   
                                                            1     448563   
            1995 39112.0 60.674587  13033.0 19350.0         0   32794998   
                                                            1     446128   
            1996 40207.0 62.386536  13644.0 20155.0         0   36609731   
...                                                                  ...   
56045       2016 7213.0  95.607864  4043.0  42585.0         1     397900   
            2017 6986.0  97.214310  3964.0  42816.0         0     777479   
                                                            1     413931   
            2018 6967.0  100.000000 4043.0  44737.0         0     836692   
                                                            1     444261   

                                                                all_loans  \
FIPS_county year pop     cpi_2018   jobs    percap_pers_inc cb              
1001        1994 38186.0 58.883015  12751.0 18683.0         0    12012117   
                                                            1      229444   
            1995 39112.0 60.674587  13033.0 19350.0         0    23395196   
                                                            1      257205   
            1996 40207.0 62.386536  13644.0 20155.0         0    25758406   
...                                                                   ...   
56045       2016 7213.0  95.607864  4043.0  42585.0         1      202250   
            2017 6986.0  97.214310  3964.0  42816.0         0      623355   
                                                            1      236390   
            2018 6967.0  100.000000 4043.0  44737.0         0      672740   
                                                            1      272249   

                                           

In [16]:
indices = list(FDIC_sb_gi.index)

In [17]:
#filtering for counties that do not have 21 years of data for both community banks and non community banks
f_index = 0
cb_index = -1

county = list(map(lambda x: x[f_index], FDIC_sb_gi.index))
cb_presence = list(map(lambda x: x[cb_index], FDIC_sb_gi.index))
df = pd.DataFrame(county)
df['1'] = cb_presence
df = df.groupby([0]).mean()
desirable_c = list(df[df["1"] == .5].index)
newIndex = [x for x in indices if x[f_index] in desirable_c]
FDIC_sb_g = FDIC_sb_gi.loc[newIndex]

percap_pers_inc = list(map(lambda x: x[-2], FDIC_sb_g.index))
FDIC_sb_g["percap_pers_inc"] = percap_pers_inc
FDIC_sb_g = FDIC_sb_g.reset_index(level = -2, drop = True)

cb = list(map(lambda x: x[-1], FDIC_sb_g.index))
FDIC_sb_g["cb"] = cb
FDIC_sb_g = FDIC_sb_g.reset_index(level = -1, drop = True)

pop = list(map(lambda x: x[2], FDIC_sb_g.index))
FDIC_sb_g["pop"] = pop
FDIC_sb_g = FDIC_sb_g.reset_index(level = 2, drop = True)

<IPython.core.display.Javascript object>

In [18]:
FDIC_sb_g = FDIC_sb_g.reset_index([0, 1, 2, 3])

In [19]:
FDIC_sb_o = FDIC_sb_g.set_index(["year","cpi_2018", "FIPS_county", "pop", "percap_pers_inc", "jobs","cb"]).sort_index(level = 0)
FDIC_sb_o

branch_count  \
year cpi_2018   FIPS_county pop      percap_pers_inc jobs    cb                 
1994 58.883015  1001        38186.0  18683.0         12751.0 0              5   
                                                             1              6   
                1003        116565.0 20129.0         50770.0 0             36   
                                                             1              7   
                1005        27751.0  15930.0         12994.0 0              3   
...                                                                       ...   
2018 100.000000 56041       20299.0  40280.0         11857.0 1              2   
                56043       7885.0   48184.0         5320.0  0              5   
                                                             1              2   
                56045       6967.0   44737.0         4043.0  0              1   
                                                             1              2   

                                                                 deposits  \
year cpi_2018   FIPS_county pop      percap_pers_inc jobs    cb             
1994 58.883015  1001        38186.0  18683.0         12751.0 0      69223   
                                                             1     131992   
                1003        116565.0 20129.0         50770.0 0     978407   
                                                             1      95509   
                1005        27751.0  15930.0         12994.0 0      48752   
...                                                                   ...   
2018 100.000000 56041       20299.0  40280.0         11857.0 1     177597   
                56043       7885.0   48184.0         5320.0  0     139289   
                                                             1      96219   
                56045       6967.0   44737.0         4043.0  0      71540   
                                                             1     144001   

                                                                     asset  \
year cpi_2018   FIPS_county pop      percap_pers_inc jobs    cb              
1994 58.883015  1001        38186.0  18683.0         12751.0 0    18428959   
                                                             1      448563   
                1003        116565.0 20129.0         50770.0 0    25949612   
                                                             1      104850   
                1005        27751.0  15930.0         12994.0 0     7796559   
...                                                                    ...   
2018 100.000000 56041       20299.0  40280.0         11857.0 1      191495   
                56043       7885.0   48184.0         5320.0  0   545996109   
                                                             1      574509   
                56045       6967.0   44737.0         4043.0  0      836692   
                                                             1      444261   

                                                                 all_loans  \
year cpi_2018   FIPS_county pop      percap_pers_inc jobs    cb              
1994 58.883015  1001        38186.0  18683.0         12751.0 0    12012117   
                                                             1      229444   
                1003        116565.0 20129.0         50770.0 0    17124846   
                                                             1       59763   
                1005        27751.0  15930.0         12994.0 0     5327730   
...                                                                    ...   
2018 100.000000 56041       20299.0  40280.0         11857.0 1       58503   
                56043       7885.0   48184.0         5320.0  0   342015041   
                                                             1      312875   
                56045       6967.0   44737.0         4043.0  0      672740   
                                                          

In [21]:
FDIC_sb_g.to_csv("FDIC_data.csv")